Simple example of eXirt's use

Run all bullets in the sequence below!



Generate files?


## Install dependences

In [1]:
!pip install openml
!pip install catsim

import openml
import pandas as pd
import statistics
import numpy as np
import random
import copy


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from google.colab import files



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 KB 4.2 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=0a9026422fe506fce21cef282771293a9d2aaf1dff0836441dbb1335efcb1d8b
  Stored in directory: /root/.cache/pip/wheels/a2/de/68/bf3972de3ecb31e32bef59a7f4c75f0687a3674c476b347c14
Successfully built liac-arff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py

## Install eXirt 

In [2]:
!pip install eXirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for eXirt: filename=eXirt-1.0.0.6-py3-none-any.whl size=8960 sha256=85fd4cee58f7948284c5755b85cb92ae00db8fe757f0b732eaa36ed2733e180b
  Stored in directory: /root/.cache/pip/wheels/73/6d/d5/fa83a2fa26568b8e30a6aa100fcd6db0b485ba319a3ddd9a08
Successfully built eXirt


In [47]:
from eXirt import eXirt

## Load data and Pre-process

In [31]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(len(df_norm[column].unique()) > 1): #fix NaN generation
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        else:
          df_norm[column] = 0
    return df_norm

In [32]:
#Select dataset name by openml link https://www.openml.org/search?sort=date
dataset_name = "pc1"

In [33]:
#load dataset by OpenML

dataset = openml.datasets.get_dataset(dataset_name)
X, Y, categorical_indicator, attribute_names = dataset.get_data(
                  dataset_format="dataframe", target=dataset.default_target_attribute)

print(dataset)
#Preprocess Y and X numerics

if (Y.dtype != 'numeric'):
  Y = Y.astype(int)

for i,c in enumerate(X.columns):
  if (X[c].dtype != 'float64'):
    X = X.astype(float)

#Normalization
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test



OpenML Dataset
Name..........: pc1
Version.......: 1
Format........: ARFF
Upload Date...: 2014-10-06 23:57:45
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/53951/pc1.arff
OpenML URL....: https://www.openml.org/d/1068
# of features.: 22
# of instances: 1109


## Creation and prediction model

In [34]:
model = RandomForestClassifier(200)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

## Global explanation rank

In [48]:
explainer = eXirt.eXirt()

In [36]:
global_explanation_attributes, global_explanation_attributes_scores = explainer.explainRankByEXirt(model, X_train, X_test, y_train, y_test,dataset_name)

In [37]:
global_explanation_attributes

['lOBlank',
 'lOComment',
 'locCodeAndComment',
 'uniq_Opnd',
 'loc',
 'I',
 'B',
 'lOCode',
 'uniq_Op',
 'L',
 'v(g)',
 'branchCount',
 'total_Opnd',
 'V',
 'T',
 'ev(g)',
 'E',
 'total_Op',
 'iv(G)',
 'D',
 'N']

In [38]:
global_explanation_attributes_scores

,Final Score
lOBlank,-114.108224
lOComment,-114.055164
locCodeAndComment,-113.910991
uniq_Opnd,-113.878039
loc,-113.873193
I,-113.817281
B,-113.718709
lOCode,-113.675764
uniq_Op,-113.650467
L,-113.650459
